In [1]:
!pip3 install torch torchvision torchaudio
!pip install numpy

In [5]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import random
import copy

In [6]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using {device} device')

Using cpu device


In [46]:

def inheritParam(child, parentList):
    with torch.no_grad():
        for layerInd, layer in enumerate(child.linear_relu_stack):
            #parentLayer = parent.linear_relu_stack[layerInd]
            if type(layer) == torch.nn.modules.linear.Linear:
                for i, row in enumerate(layer.weight):
                    for j, val in enumerate(row):
                        parent = random.choice(parentList)
                        parentLayer = parent.linear_relu_stack[layerInd]
                        layer.weight[i][j] = parentLayer.weight[i][j]
                for i, val in enumerate(layer.bias):
                    parent = random.choice(parentList)
                    parentLayer = parent.linear_relu_stack[layerInd]
                    layer.bias[i] = parentLayer.bias[i]
                #print(layer.weight, layer.bias)
                #print(parentLayer.weight, parentlayer.bias)
            
def mutateParam(model, mutRate, mutAmp):
    with torch.no_grad():
        for layer in model.linear_relu_stack:
            if type(layer) == torch.nn.modules.linear.Linear:
                for i, row in enumerate(layer.weight):
                    for j, val in enumerate(row):
                        reroll = random.random()
                        if reroll <= mutRate:
                            row[j] = val + random.uniform(-mutAmp, mutAmp)
                for i, val in enumerate(layer.bias):
                    reroll = random.random()
                    if reroll <= mutRate:
                        layer.bias[i] = val + random.uniform(-mutAmp, mutAmp)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(3, 16),
            nn.ReLU(),
            nn.Linear(16, 8),
            nn.ReLU(),
            nn.Linear(8, 4),
            nn.ReLU(),
            nn.Linear(4, 2),
            nn.ReLU(),
            nn.Linear(2, 1),
        )
        self.fitness = 0

    def forward(self, x):
        x = self.flatten(x)
        output = self.linear_relu_stack(x)
        return output
    
    def setFitness(self, fitness):
        self.fitness = fitness

#model = NeuralNetwork().to(device)


class Farm:
    def __init__(self, initSize):
        self.size = initSize
    
    def grow(self):
        self.size *= 1.5
    
    def harvest(self, num):
        originalSize = self.size
        if num < 0:
            return 0
        elif num > self.size:
            num = self.size
            self.size -= num
            return originalSize
        else:
            self.size -= num
            return num
        
class City:
    def __init__(self, initSize):
        self.size = initSize
    
    def gather(self, farm, num):
        if num < 0:
            num = 0
        elif num > self.size:
            num = self.size
        numGathered = farm.harvest(num)
        self.size += numGathered / 2


def simulateCity(model):
    city = City(10)
    farm = Farm(10)
    years = 10

    avgGathered = 0
    for year in range(1, years):
        dataInput = torch.tensor([[float(city.size), float(farm.size), float(year)]])
        #dataInputNorm = torch.nn.functional.normalize(dataInput)                        

        numGathered = float(model(dataInput)) * 1
        #print(numGathered)
        avgGathered += numGathered
        city.gather(farm, numGathered)
        farm.grow()
    #print(farm.size)
    return city.size

class nnPopulation:
    def __init__(self, batchSize):
        self.population = [NeuralNetwork().to(device) for x in range(batchSize)]
        self.batchSize = batchSize
    
    def displayBest(self, num):
        self.population.sort(key = lambda x:x.fitness, reverse=True)
        for model in self.population[:num]:
            print("Fitness:", model.fitness)
        
    def runBatch(self):
        for model in self.population:
            model.setFitness(simulateCity(model))
    
    def getFittest(self, cutRate):
        fittest = copy.deepcopy(self.population)
        fittest.sort(key = lambda x:x.fitness, reverse=True)
        numRemaining = round(self.batchSize * cutRate)
        fittest = fittest[:numRemaining]
        return fittest
    
    def repopulate(self, cutRate):
        self.parents = self.getFittest(cutRate)
        self.population.sort(key = lambda x:x.fitness, reverse=True)
        self.population = self.population[:round(self.batchSize * cutRate)]
        while len(self.population) < self.batchSize:
            if len(self.population) % 100 == 0:
                print(len(self.population))
            child = NeuralNetwork().to(device)
            inheritParam(child, self.parents)
            mutateParam(child, 0.05, 0.10)
            self.population.append(child)

In [47]:
village = nnPopulation(500)
#generations = 10

for x in range(200):
    print("Generation:", x)
    village.runBatch()
    village.displayBest(5)
    village.repopulate(0.10)



Generation: 0
Fitness: 26.18032658100128
Fitness: 24.937258183956146
Fitness: 21.767029285430908
Fitness: 21.479845374822617
Fitness: 20.00757548213005
100
200
300
400
Generation: 1
Fitness: 32.88014280796051
Fitness: 27.28727436065674
Fitness: 27.206638425588608
Fitness: 26.271910548210144
Fitness: 26.265959322452545
100
200
300
400
Generation: 2
Fitness: 32.88014280796051
Fitness: 31.168783247470856
Fitness: 31.03739833831787
Fitness: 29.704929769039154
Fitness: 29.623465299606323
100
200
300
400
Generation: 3
Fitness: 35.01220527291298
Fitness: 33.21214661002159
Fitness: 32.88014280796051
Fitness: 32.55780303478241
Fitness: 31.851516485214233
100
200
300
400
Generation: 4
Fitness: 35.78332868218422
Fitness: 35.03648257255554
Fitness: 35.01220527291298
Fitness: 34.82555013895035
Fitness: 34.74796634912491
100
200
300
400
Generation: 5
Fitness: 38.750414699316025
Fitness: 36.84622247517109
Fitness: 35.96544370055199
Fitness: 35.78332868218422
Fitness: 35.69989696145058
100
200
300
400

In [60]:
def simulateCity(model):
    city = City(10)
    farm = Farm(10)
    years = 10

    avgGathered = 0
    print(f"City Size:\tFarm Size:\tNum Gathered:")
    for year in range(1, years):
        dataInput = torch.tensor([[float(city.size), float(farm.size), float(year)]])
        #dataInputNorm = torch.nn.functional.normalize(dataInput)                        

        numGathered = float(model(dataInput)) * 1
        avgGathered += numGathered
        city.gather(farm, numGathered)
        farm.grow()
        print(f"{round(city.size, 2)}\t\t{round(farm.size, 2)}\t\t{round(numGathered, 2)}")
    #print(farm.size)
    return city.size

dir(village)
#village.displayBest(10)
bestModel = village.population[0]
simulateCity(bestModel)

City Size:	Farm Size:	Num Gathered:
10.0		15.0		-0.59
10.0		22.5		-0.71
10.0		33.75		-0.71
10.0		50.62		-0.36
14.81		61.51		9.62
22.21		70.06		20.74
33.32		71.77		31.77
49.98		57.67		42.31
74.97		11.55		50.78


74.96604359149933